This code will create a STAC catalog by crawling over folders in the Cyverse Data Store and looking for geospatial assets (e.g., orthomosaics, point clouds, DEMs). 

Please launch 'JupyterLab_Geospatial' VICE app in Cyverse Discovery Environment
Open this Jupyter Notebook `STAC_creation_latest.ipynb`
Change the Kernel to `osgeo`
Use at least 16gb of RAM

In [1]:
#Install python libraries used in this script
%pip install rasterio==1.3.8 shapely==2.0.1 pystac==1.8.4 piexif==1.1.3 pandas==2.0.2 geopy==2.4.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
########Import libraries, modules, and classes into environment. ########
###########################

#Import the Path class from the pathlib module in Python. \
#The pathlib module is part of the Python Standard Library and provides \
#an object-oriented way to work with file system paths, \
#making it more convenient and readable than using traditional string-based file paths.

#The Path class is a high-level and flexible representation of file system paths. \
#It can be used for various purposes, such as constructing file system paths, navigating directories, \
#creating directories, reading or writing files, and more.
from pathlib import Path

#The os module is part of the Python Standard Library and provides a \
#collection of functions for interacting with the operating system. 
#This module allows you to perform various tasks such as creating, deleting, or modifying files and directories, \
#obtaining system information, and managing processes.
import os

#Import Optional typing hint
from typing import Optional

#ThreadPoolExecutor is a class from the 'concurrent.futures' module which comes from the Standard Python Library. 
#It provides a high level interface for asynchronously executing functions using threads. It can be used to execute \
#a certain number of functions in parallel, leveraging multiple threads. 
from concurrent.futures import ThreadPoolExecutor


#Import the pystac library into environment. Pystac (https://pystac.readthedocs.io/en/stable/index.html)
#is a library for reading and writing STAC stuff
import pystac 

#ProjectionExtension is a class within the module 'pystac.extensions.projection'. \
#We are using it to show the map projection of raster dataset
from pystac.extensions.projection import ProjectionExtension

#RasterExtension is a class within the module 'pystac.extensions.raster'
#we are using it to display various info about the raster
from pystac.extensions.raster import RasterExtension


from pystac.extensions.scientific import ScientificExtension
from pystac.extensions.scientific import Publication

from pystac.extensions.raster import RasterExtension


from pystac.provider import Provider
from pystac.provider import ProviderRole
import pystac.provider 
from pystac import Provider


#rasterio is a library to read and write raster data. It will be used in this script to extract information \
#from drone imagery geotiffs 
import rasterio
import rasterio.warp #used to convert coordinate of a raster bounding box

#Import the pdal lirbary into the environment. We use pdal (https://pdal.io/en/latest/) for reading LAS and LAZ files
#pdal is installed in the osgeo kernel, so we do not 'pip install pdal' in this script
import pdal

# Used to assist with pdal work
from osgeo import osr
from osgeo import ogr
from geopy.distance import geodesic


#Used to create a polygon of the raster bounds
from shapely.geometry import Polygon, mapping

#For creating a geojson output file of STAC items. This module comes in the Python Standard Library
import json

# datetime is a module in the Python's standard library. We need it to assign collection dates to each item or asset
from datetime import datetime

#We will use pandas to bring a csv file into a dataframe
import pandas

#We use the subprocess module when processing LAS/LAZ data. This prevents the kernel from \
#falsely getting flagged as unresponsive and automatically restarted (causing all work to be lost)
import subprocess

In [3]:
######This cell is for users to input manual metadata for imagery assets that will become STAC catalogs.#######
############################

#source_image_folder = '/data-store/iplant/home/jgillan/stac_test'
#source_image_folder = '/data-store/iplant/home/shared/commons_repo/curated/Gillan_Ecosphere_2021/raster_products/May_2019'
source_image_folder = '/data-store/iplant/home/shared/ofo/internal/david_data_for_Jeff/3_geometric-derived/metashape/reconstuction-000000/exports'

# output folder (will overwrite existing files with matching output names)
#stac_output_directory = '/data-store/iplant/home/jgillan/STAC_drone/SRER_May2019'
#stac_output_directory = '/data-store/iplant/home/jgillan/stac_test/oct9'
stac_output_directory = '/data-store/iplant/home/shared/ofo/internal/david_data_for_Jeff/3_geometric-derived/metashape/reconstuction-000000/exports/stac_test'

# Additional Metadata to add to items
platform = 'DJI Phantom 4 RTK'
license = 'CC-BY-SA-4.0'
items_mission_description = 'Open Forest Observatory Test Data'
pub_doi = '10.1002/ecs2.3649'
citation = 'Gillan, JK., GE Ponce-Campos, TL Swetnam, A Gorlier, P Heilman, MP McClaran. 2021. Innovations to expand drone data collection and analysis for rangeland monitoring. Ecosphere, 12(7)'

# collection definitions
collection_id = 'ofo_test' # needs to be folder-name compatible
collection_description = 'The imagery was part of the Ecostate Mapping project of 2019 at Santa Rita Experimental Range'

# top-level catalog definitions
catalog_id = 'Cyverse Remotely Sensed Imagery STAC Catalog'
catalog_description = 'This catalog includes all of the imagery assets the exist in Cyverse Data Store'


# create default datetime object for the collection - used when all items were collected on same date
default_datetime = datetime(year=2023, month=5, day=25, hour=12)

#If items were collected on different dates, then you should supply a csv that has the following columns:'Id', 'collection_date'
#df_collection_date = pandas.read_csv('/data-store/iplant/home/jgillan/stac_test/collection_date.csv')
#df_collection_date = pandas.read_csv('/data-store/iplant/home/shared/commons_repo/curated/Gillan_Ecosphere_2021/raster_products/May_2019/srer_may2019.csv')
#df_collection_date = pandas.read_csv('/data-store/iplant/home/shared/ofo/internal/david_data_for_Jeff/3_geometric-derived/metashape/reconstuction-000000/exports/collection_date.csv')
df_collection_date = pandas.read_csv('/data-store/iplant/home/jgillan/collection_date.csv')
                                                 
#Provider information
provider = Provider(name='Derek Young',
                       description="A provider that supplies example geospatial data.",
                       roles=[ProviderRole.PRODUCER, ProviderRole.PROCESSOR],
                       url='https://openforestobservatory.org/')

provider_dict = provider.to_dict()

#Email contact variable
contact_email = 'djyoung@ucdavis.edu'


# email host variables - DON'T CHANGE these unless you know what you're doing
smtp_host = '128.196.254.80'

In [4]:
#########Functions to extract the spatial resolution(ground sampline distance) from geotif imagery products \
##########These functions are using the 'rasterio' library.

# function to truncate spatial resolution to 3 decimal places
def trun_n_d(num,n):
    num_s = str(num)
    if 'e' in num_s or 'E' in num_s:
        return '{0:.{1}f}'.format(num,n)
    i,p,d = num_s.partition('.')
    return '.'.join([i,(d+'0'*n)[:n]])

# function to get the spatial resolution of a raster. It only works if the imagery products has a map projection, otherwise 
#it will return 0.00 
def spatial_resolution(raster):
    """extracts the XY Pixel Size"""
    t = raster.transform
    x = t[0]
    y = -t[4]
    x_trunc = trun_n_d(x, 3)
    y_trunc = trun_n_d(y, 3)
    return x_trunc, y_trunc

In [5]:
###########Function to calculate the bounding box and footprint of a geospatial raster dataset
######################

#This creates a function called 'get_bbox_and_footprint' for a raster we are calling 'dataset'
def get_bbox_and_footprint(dataset):

    # extract the bounding box of a raster using rasterio. '.bounds' is an attribute of 'rasterio.DatasetReader'
    #'bounds' returns the left, bottom, right, and top coordinates of a raster
    bounds = dataset.bounds
    
    #Transform the coordinate system of the raster bounds from it's orginial coordinate reference system \
    #to wgs84 which is also known as EPSG 4326. It uses the rasterio submodule 'rasterio.warp'
    bounds = rasterio.warp.transform_bounds(dataset.crs, 'EPSG:4326', 
                                            bounds.left, bounds.bottom, bounds.right, bounds.top)
   
    #The transformed bounds are then used to create a new rasterio.coords.BoundingBox object. \
    #The rasterio.coords.BoundingBox class is a convenient way to represent a bounding box \
    #with named attributes (left, bottom, right, and top) instead of using a tuple or a list. \
    #This makes the code more readable and easier to work with.
    bounds = rasterio.coords.BoundingBox(bounds[0], bounds[1], bounds[2], bounds[3])
    
    
    #The isinstance() function checks if the bounds variable is an instance of the \
    #rasterio.coords.BoundingBox class. If it is, it means that the bounds variable \
    #represents a bounding box with named attributes (left, bottom, right, and top).
    #If the bounds variable is an instance of rasterio.coords.BoundingBox, \
    #the code creates a list called bbox, which contains the bounding box coordinates \
    #in the following order: left, bottom, right, and top. This list is a more straightforward \
    #way to represent the bounding box as a sequence of coordinates.
    if isinstance(bounds, rasterio.coords.BoundingBox):
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        
    #If the bounds variable is not an instance of rasterio.coords.BoundingBox, \
    #the code assumes that it is a callable object (such as a function or a method) \
    #that returns the bounding box coordinates when called. In this case, the code \
    #calls the bounds() function and converts the returned bounding box coordinates \
    #to a list of floating-point numbers. This list is then assigned to the bbox variable.    
    else:
        bbox = [float(f) for f in bounds()]

    # create vertices and polygon from the bounding box coordinates. It uses the 'shapely.geometry' module \
    #from the shapely library. The output is the'footprint' variable which contains a shapely.geometry.Polygon object \
    #that represents the footprint of the raster dataset, based on its bounding box coordinates. \
    #This footprint can be used for tasks like spatial analysis, visualization, or overlaying with other geospatial data.
    footprint = Polygon([
        [bbox[0], bbox[1]],#left bottom
        [bbox[0], bbox[3]],#left top
        [bbox[2], bbox[3]],#right top
        [bbox[2], bbox[1]] #right bottom
    ])
    
    #Return the calculated bbox and the footprint as a dictionary using the mapping function \
    #(from the shapely.geometry module)
    return bbox, mapping(footprint)

In [6]:
##Function to get the spatial information on tiff imagery products

def tif_get_spatial_info(tif_file_path: Path) -> Optional[tuple]:

    # Open the individual file with rasterio
    ds = rasterio.open(tif_file_path)

    # Apply the function to get the bounding box (left, bottom, right, top) and make a footprint rectangle
    bbox, footprint = get_bbox_and_footprint(ds)

    # Extract the spatial resolution (gsd) of the image product using the function 'spatial_resolution'.
    x_res, y_res = spatial_resolution(ds)

    # Return the path, bounding box, the footprint, and the X and Y GSD (Ground Sample Distance)
    return tif_file_path, bbox, footprint, x_res, y_res, ds.shape, ds.crs.to_epsg(), pystac.MediaType.COG

In [7]:
#Determines the GSD (ground sampling distance) of an LAS/LAZ file

def las_spatial_resolution(bounding_box: tuple, num_points: int) -> tuple:
    
    min_y, min_x, max_y, max_x = bounding_box

    gsd_x = geodesic((min_x, min_y), (max_x, min_y)).meters / float(num_points)
    gsd_y = geodesic((min_x, min_y), (min_x, max_y)).meters / float(num_points)
    
    return gsd_x, gsd_y

In [8]:
####Returns the bounding box, footprint, and EPSG of the passed in LAS/LAZ file content

def get_las_bbox_footprint(las_srs_wkt: str, boundary_wkt: str, target_epsg: int=4326) -> list:
    # Returns a list containing the bounding box, and footprint geometries
    
    bounds_polygon = ogr.CreateGeometryFromWkt(boundary_wkt)

    # Translate the points if needed
    src_srs = osr.SpatialReference()
    src_srs.ImportFromWkt(las_srs_wkt)

    geom_epsg = int(src_srs.GetAttrValue('AUTHORITY', 1))
    if geom_epsg != target_epsg:
        # Set up the transformation
        dst_srs = osr.SpatialReference()
        dst_srs.ImportFromEPSG(target_epsg)

        transform = osr.CoordinateTransformation(src_srs, dst_srs)
        
        # Perform the transformation
        transl_polygon = bounds_polygon.Clone()
        transl_polygon.Transform(transform)
        
        bounds_polygon = transl_polygon

    # Get the envelope of the polygon and re-arrange to expected order
    return_bbox = bounds_polygon.GetEnvelope()
    return_bbox = (return_bbox[2], return_bbox[0], return_bbox[3], return_bbox[1])

    # Create the polygon of the footprint
    footprint = Polygon([
        [return_bbox[0], return_bbox[1]],#left bottom
        [return_bbox[0], return_bbox[3]],#left top
        [return_bbox[2], return_bbox[3]],#right top
        [return_bbox[2], return_bbox[1]] #right bottom
    ])

    return return_bbox, footprint, geom_epsg


In [9]:
####Function to get the spatial information on the point cloud files

#Return the information on the LAS/LAZ files
def las_get_spatial_info(las_file_path: Path) -> Optional[tuple]:

    # Folder to write temporary files to
    root_dir = '/tmp'
        
    # Configure the processing pipeline for the data we will want
    pipeline_json  = """
    {
        "pipeline": [
            "%s",
            {
                "type" : "filters.hexbin"
            }
        ]
    }
    """ % las_file_path

    # Get the filenames for the pipeline text, and the output of the pipeline
    base_filename, file_ext = os.path.splitext(os.path.basename(las_file_path))
    pipeline_path = os.path.join(root_dir, base_filename + '_pipeline.json')
    out_path = os.path.join(root_dir, base_filename + '.json')
    with open(pipeline_path, "w") as outfile:
        outfile.write(pipeline_json)

    # Make the call to run the pdal app to process the pipeline
    result = subprocess.run(['/opt/conda/envs/osgeo/bin/pdal', 'pipeline', pipeline_path, '--metadata', out_path], stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE)

    # Load the output of the pipeline
    with open(out_path, 'r') as infile:
        metadata = json.load(infile)

    # Clean up the temporary files
    os.remove(pipeline_path)
    os.remove(out_path)

    # Assign to variable to make it easier for a developer
    readers = metadata['stages']['readers.las']
    hexbin = metadata['stages']['filters.hexbin']

    # Get the bounding box in the correct coordinate system
    bbox, footprint, epsg = get_las_bbox_footprint(readers['srs']['wkt'], hexbin['boundary'])

    # Extract the spatial resolution (gsd) of the image product using the function 'las_spatial_resolution'.
    x_res, y_res = las_spatial_resolution(bbox, readers['count'])

    # Return the path, bounding box, the footprint, and the X and Y GSD (Ground Sample Distance)
    return las_file_path, bbox, mapping(footprint), x_res, y_res, (None, None), epsg, 'application/vnd.laszip' if file_ext == '.laz' else 'application/vnd.las'

In [10]:

###This starts the for loop to crawl through a folder, find imagery assets, and generate STAC json metadata files for each item

os.environ['PROJ_DATA'] = '/opt/conda/envs/osgeo/share/proj'

# Disable a warning message
ogr.UseExceptions()

# Get a list of geospatial files (in this case located in Cyverse data store)
folder = Path(source_image_folder)


# Create an extension mapping dictionary to assist with the file discovery and multithreading
# Each extension maps to the function to call
ext_map = {'.tif': tif_get_spatial_info,
           '.las': las_get_spatial_info,
           '.laz': las_get_spatial_info
          }

# Load the files of interest
files = list()
for one_filter in ext_map.keys():
    files = files + list(folder.rglob('*' + one_filter))

# Initialize variables to hold the results
items_dict = {}
all_items = []

# Create a ThreadPoolExecutor with 2 worker threads to load all the spatial data
with ThreadPoolExecutor(max_workers=2) as executor:
    # Submit each file to the executor using the file extension to determine which function to call
    future_results = [executor.submit(ext_map[os.path.splitext(file)[1]], file) for file in files]

    # Get the results as they become available
    all_results = [future.result() for future in future_results]

    
# Loop through each item in the folder and do several things
for result in all_results:

    #Get the specific results from processing one file
    file, bbox, footprint, x_res, y_res, width_height, srid, media_type = result

    # the ID (name) for each indivual file
    idx = file.stem
    
    
    
    ##The following 3 lines are for assigning the date of imagery collection to each of the item by matching IDs from a csv file
    
    
    # Within the for loop that we are in, this line looks at the 'Id' column of the csv file (imported into python as a pandas DataFrame).
    # It takes the Id name of the current geotiff file and looks for a match within the Id column within 'df_collection_date'. 
    # If it finds a matching ID, it returns the info for the entire row. IDs in the dataframe are stored as strings. 
    
    collection_time = df_collection_date[df_collection_date.Id == str(idx)]
    
    # From the matched row, this command will return the value within the 'collection_date' column as a 2D numpy array 
    dates = collection_time.collection_date.values
    
    # Get the plot name of the item
    plot = collection_time['plot'].iloc[0]
    
    # Convert the date into a 'datetime' object (e.g., 2019-05-19)
    datess = datetime.strptime(dates[0], '%Y-%m-%d')

    # Check if an item with this plot name already exists
    if plot in items_dict:
        #if it does, get that item
        item = items_dict[plot]
        
    # If this is an item with a new plot name 
    else:    
        # create a STAC item for each individual file 
        item = pystac.Item(id=plot,
                geometry=footprint,
                bbox=bbox,
                datetime=datess,
                #datetime=default_datetime,
                stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json',
                                 'https://stac-extensions.github.io/scientific/v1.0.0/schema.json'],
                 
                properties={'gsd': x_res,
                        'platform': platform,
                        'license': license,
                        'mission': items_mission_description,
                        'sci:doi': pub_doi,
                        'sci:citation': citation,
                        'providers': [provider_dict]}
        )
    items_dict[plot] = item
    
    # Adding the map projection extension to each item. Otherwise, the projection info will not display 
    params = {
        'bbox': bbox,
        'geometry': footprint
        }
    if None not in width_height:
        params['shape'] = width_height  #ds.shape,
    ProjectionExtension.ext(item).apply(srid, #ds.crs.to_epsg(),
                                        **params)
                                        #transform = [float(getattr(ds.transform, letter)) for letter in 'abcdef']
                                        #)
   
    # Add the asset link to the item and define the type of geospatial format it is
    item.add_asset(
        key=idx,
        asset=pystac.Asset(
            href=file.as_posix(), 
            media_type=media_type, #pystac.MediaType.COG
            roles='data'
            #extra_fields=asset_ext
        )
    )

    # Add each STAC item to a list of all the items
    all_items.append(item)
    all_items = list(items_dict.values())

print('Done processing files')

Done processing files


In [11]:
#######Create and describe a STAC Collection 
##########################

# the geographic extent of all the items added
item_extents = pystac.Extent.from_items(all_items)

# creating the collection
collection = pystac.Collection(id=collection_id,
                               description=collection_description,
                               extent=item_extents,
                               license=license)

# add all STAC item to the STAC collection
for item in all_items:
    collection.add_item(item)

In [12]:
# print the number of STAC items that were added to the STAC catalog    
print(len(list(collection.get_items())))

# describe the items in the STAC catalog
collection.describe()

1
* <Collection id=ofo_test>
  * <Item id=15_g2>


In [13]:
##########Save the collection and its child items to output directory (ABSOLUTE LINKS FOR STAC API)

collection.normalize_hrefs(stac_output_directory)

collection.save(catalog_type=pystac.CatalogType.ABSOLUTE_PUBLISHED)


In [14]:
#########Combine all of the STAC item metadata into a single geojson and output to directory
####################

# Create an empty GeoJSON FeatureCollection
geojson = {
    "type": "FeatureCollection",
    "features": []
}

# Iterate through all the items in the collection and convert them to GeoJSON Features
for item in collection.get_all_items():
    # Get the STAC Item as a dictionary
    item_dict = item.to_dict()

    # Convert the STAC Item to a GeoJSON Feature
    feature = {
        "type": "Feature",
        "collection": item_dict["collection"],
        "stac_version": item_dict["stac_version"],
        "stac_extensions": item_dict["stac_extensions"],
        "id": item_dict["id"],
        "geometry": item_dict["geometry"],
        "bbox": item_dict["bbox"],
        "properties": item_dict["properties"],
        "assets": item_dict["assets"]
    }

    # Add the GeoJSON Feature to the FeatureCollection
    geojson["features"].append(feature)


# Make sure the output_directory ends with a path separator
if not stac_output_directory.endswith("/"):
    stac_output_directory += "/"

output_file_path = stac_output_directory + "index.geojson"

# Write the GeoJSON FeatureCollection to a file
with open(output_file_path, "w") as f:
    json.dump(geojson, f, indent=4)        

    

# Define the string to find and the string to replace it with
string_to_find = "/data-store"
string_to_replace = "https://data.cyverse.org/dav-anon"

# Read the file into a string
with open(output_file_path, "r") as f:
    file_content = f.read()

# Perform the find-and-replace operation
file_content = file_content.replace(string_to_find, string_to_replace)

# Write the result back to the file
with open(output_file_path, "w") as f:
    f.write(file_content)
    
    
   
    

To make the saved catalog available, you will need to:
-  share the output folder to \"public\" using the [Discovery Environment](https://de.cyverse.org/) Data tab
- add your catalog to the CyVerse [master catalog](/iplant/home/jgillan/stac.cyverse.org/cyverse_stac_catalog/catalog.json)


In [15]:
#%pip install python-irodsclient

The following will share the folder containing your catalog with the CyVerse `public` user using read-only permissions

You will be prompted to enter your CyVerse username and password. Be sure to use the *Enter* key for each prompt

In [16]:
#import os
#from getpass import getpass
#from irods.access import iRODSAccess
#from irods.session import iRODSSession

#irods_username = input('Enter your CyVerse user name:')
#irods_password = getpass('Enter your CyVerse password:')

#sess = iRODSSession(host='data.cyverse.org', port=1247, user=irods_username, password=irods_password, zone='iplant')

#user = sess.users.get(sess.username, sess.zone)

# add the needed users for linking
#acl_path = '/' + os.path.join(*(stac_output_directory.split(os.path.sep)[2:]))
#acl = iRODSAccess('read', acl_path, 'anonymous', user.zone)
#sess.acls.set(acl, recursive=True)

#sess.cleanup()

#print("Folder updated")

**Almost Done!**

To have a *newly created* catalog added to the global CyVerse catalog, update `your_email` using your email address
and send an email using the following:

In [17]:
#your_email = '<someone>@arizona.edu'

# Import smtplib for the actual sending function
#import smtplib

# Import the email modules we'll need
#from email.mime.text import MIMEText

#msg = MIMEText("Please add the following STAC catalog to the main CyVerse catalog: " + 
               #"https://data.cyverse.org/dav-anon/" + os.path.join(*(stac_output_directory.split(os.path.sep)[2:])) +
               #"/catalog.json")
#msg['Subject'] = 'Add new STAC catalog to main CyVerse catalog'
#msg['From'] = your_email
#msg['To'] = contact_email

# Send the message via our own SMTP server, but don't include the
# envelope header.
#smtp = smtplib.SMTP(smtp_host)
#smtp.sendmail(your_email, [contact_email], msg.as_string())
#smtp.quit()


In [1]:
# Install all dependencies with specific versions
%pip install \
  geopandas \
  shapely==1.8.5 \
  rasterio==1.3.4 \
  laspy==2.2.0 \
  pystac==1.6.0 \
  pyproj==3.4.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 151.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.9/615.9 kB 155.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 187.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 130.7 MB/s eta 0:00:0000:0100:01
INFO: pip is looking at multiple versions of geopandas to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.4/

In [1]:
import os
from pathlib import Path
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, mapping
import numpy as np

# Raster / LAZ imports
import rasterio
from rasterio.warp import transform_bounds
from rasterio.coords import BoundingBox
from osgeo import ogr, osr
import laspy

# Optional: pystac for building STAC objects more systematically
import pystac
from pystac import (CatalogType, MediaType)
from pystac.extensions.eo import EOExtension
from pystac.extensions.projection import ProjectionExtension
from datetime import datetime

In [2]:
def trun_n_d(num, n):
    """
    Truncate spatial resolution to 'n' decimal places.
    """
    num_s = str(num)
    if 'e' in num_s or 'E' in num_s:
        return f'{num:.{n}f}'
    i, p, d = num_s.partition('.')
    return '.'.join([i, (d + '0'*n)[:n]])

In [3]:
def spatial_resolution(raster):
    """
    Extracts the XY Pixel Size from a rasterio Dataset.
    Only works if imagery has a map projection, otherwise returns (0.00, 0.00).
    """
    t = raster.transform
    x = t[0]
    y = -t[4]
    x_trunc = trun_n_d(x, 3)
    y_trunc = trun_n_d(y, 3)
    return x_trunc, y_trunc

In [4]:
def get_bbox_and_footprint(dataset):
    """
    Returns:
      bbox: [left, bottom, right, top] in EPSG:4326
      footprint: shapely polygon geometry (as a geo-interface dict) in EPSG:4326
    """
    # Original bounding box in native CRS
    bounds = dataset.bounds
    
    # Transform bounds to EPSG:4326
    bounds_4326 = transform_bounds(dataset.crs, 'EPSG:4326',
                                   bounds.left, bounds.bottom, 
                                   bounds.right, bounds.top)
    # Convert to a named bounding box
    bounds_4326 = BoundingBox(*bounds_4326)
    
    # Build a simple list
    bbox = [bounds_4326.left, bounds_4326.bottom, bounds_4326.right, bounds_4326.top]
    
    # Create a polygon footprint
    footprint = Polygon([
        (bbox[0], bbox[1]),  # left, bottom
        (bbox[0], bbox[3]),  # left, top
        (bbox[2], bbox[3]),  # right, top
        (bbox[2], bbox[1])   # right, bottom
    ])
    return bbox, mapping(footprint)

In [5]:
def tif_get_spatial_info(tif_file_path: Path):
    """
    Extract bounding box, footprint, resolution, CRS, shape, etc. from a GeoTIFF
    in a single function.
    Returns a dict of relevant properties or None if error.
    """
    try:
        with rasterio.open(tif_file_path) as ds:
            bbox, footprint = get_bbox_and_footprint(ds)
            x_res, y_res = spatial_resolution(ds)
            
            # Raster shape
            height, width = ds.shape
            # Attempt to retrieve EPSG integer if possible
            epsg_code = ds.crs.to_epsg() if ds.crs else None
            
            return {
                "path": str(tif_file_path),
                "bbox": bbox,
                "footprint": footprint,
                "x_res": x_res,
                "y_res": y_res,
                "width": width,
                "height": height,
                "epsg": epsg_code,
                "media_type": MediaType.COG  # Or 'image/tiff'
            }
    except Exception as e:
        print(f"Error reading {tif_file_path} - skipping. Error: {e}")
        return None

In [6]:
def get_las_bbox_footprint(las_srs_wkt: str, boundary_wkt: str, target_epsg: int = 4326):
    """
    Returns bounding box, footprint geometry, and EPSG for a LAZ file.
    
    boundary_wkt: WKT polygon of the LAZ boundary in native CRS.
    las_srs_wkt: The coordinate system WKT.
    
    Returns:
      return_bbox: (minX, minY, maxX, maxY) in EPSG:4326
      footprint: shapely Polygon geometry in EPSG:4326
      geom_epsg: original EPSG code (if found)
    """
    bounds_polygon = ogr.CreateGeometryFromWkt(boundary_wkt)

    # Parse source SRS
    src_srs = osr.SpatialReference()
    src_srs.ImportFromWkt(las_srs_wkt)

    geom_epsg = None
    try:
        geom_epsg = int(src_srs.GetAttrValue('AUTHORITY', 1))
    except:
        pass  # Not all LAS have an authority code

    # If needed, transform to target EPSG
    if geom_epsg != target_epsg:
        dst_srs = osr.SpatialReference()
        dst_srs.ImportFromEPSG(target_epsg)

        transform = osr.CoordinateTransformation(src_srs, dst_srs)
        transl_polygon = bounds_polygon.Clone()
        transl_polygon.Transform(transform)
        bounds_polygon = transl_polygon

    # Envelope -> (minX, maxX, minY, maxY)
    envelope = bounds_polygon.GetEnvelope()
    # Re-arrange to (minX, minY, maxX, maxY)
    return_bbox = (envelope[0], envelope[2], envelope[1], envelope[3])

    # Construct shapely polygon from that envelope
    footprint = Polygon([
        (return_bbox[0], return_bbox[1]),  # left, bottom
        (return_bbox[0], return_bbox[3]),  # left, top
        (return_bbox[2], return_bbox[3]),  # right, top
        (return_bbox[2], return_bbox[1])   # right, bottom
    ])

    return return_bbox, mapping(footprint), geom_epsg

In [7]:
def las_get_spatial_info(laz_file_path: Path):
    """
    Extract bounding box, footprint, approximate resolution, etc. from a LAZ file.
    Returns a dict or None on error.
    """
    try:
        # Open LAZ using laspy
        with laspy.open(laz_file_path) as f:
            header = f.header
            # bounding box in native CRS from LAS header
            minx, maxx = header.mins[0], header.maxs[0]
            miny, maxy = header.mins[1], header.maxs[1]
            # You can also check the 'z' dimension if needed

            # We'll attempt to get the WKT from the header
            las_srs_wkt = header.vlr().coordinate_system_description
            # Fallback if no WKT found
            if not las_srs_wkt:
                # If no SRS, skip or assume EPSG:4326
                las_srs_wkt = osr.SpatialReference()
                las_srs_wkt.ImportFromEPSG(4326)
                las_srs_wkt = las_srs_wkt.ExportToWkt()

            # Build WKT polygon from bounding box
            wkt_poly = f"POLYGON (({minx} {miny}, {minx} {maxy}, {maxx} {maxy}, {maxx} {miny}, {minx} {miny}))"
            bbox_4326, footprint_4326, geom_epsg = get_las_bbox_footprint(
                las_srs_wkt=las_srs_wkt,
                boundary_wkt=wkt_poly,
                target_epsg=4326
            )
            
            # Approx resolution can be guessed or omitted. 
            # E.g., many point clouds don't have a single GSD. We'll omit advanced logic,
            # or just store "N/A".
            # If you really want a rough GSD, you'd have to define your approach. 
            # We'll do "N/A" for now or store them as strings.
            x_res, y_res = "N/A", "N/A"

            return {
                "path": str(laz_file_path),
                "bbox": list(bbox_4326),  # Convert to list
                "footprint": footprint_4326,
                "x_res": x_res,
                "y_res": y_res,
                "epsg": geom_epsg if geom_epsg else 4326,
                "media_type": "application/octet-stream"  # Or "application/las" if you prefer
            }

    except Exception as e:
        print(f"Error reading {laz_file_path} - skipping. Error: {e}")
        return None

In [8]:
# 3.1 Define your paths
gpkg_path = Path("/data-store/iplant/home/shared/ofo/public/metadata/all-mission-polygons-w-metadata.gpkg")
missions_root = Path("/data-store/iplant/home/shared/ofo/public/missions")

# 3.2 Read the GPKG into a GeoDataFrame
#     We assume geometry is already EPSG:4326
gdf = gpd.read_file(gpkg_path)
print(f"Read {len(gdf)} missions from GPKG.")

Read 440 missions from GPKG.


In [9]:
gdf

,dataset_id,flight_speed_derived,flight_terrain_correlation_derived,camera_pitch_derived,smart_oblique_derived,earliest_date_derived,earliest_datetime_local_derived,latest_datetime_local_derived,single_date_derived,earliest_time_local_derived,...,altitude_ahp_obs,dataset_id_old,date,sensor_name,flight_planner_name,addl_dataset_ids_baserow,addl_baserow_differ_by,why_not_separable,smart_oblique,geometry
0,000421,3.00,0.98,0.0,False,2019-06-04,2019-06-04 11:59:40+00:00,2019-06-04 14:31:00+00:00,True,11:59:40,...,None,None,2019-06-04,P4A integrated camera,Map Pilot,None,None,None,None,"MULTIPOLYGON (((-121.06144 40.13898, -121.0608..."
1,000422,3.13,0.98,0.0,False,2019-06-06,2019-06-06 11:22:42+00:00,2019-06-06 14:30:16+00:00,True,11:22:42,...,None,None,2019-06-06,P4A integrated camera,Map Pilot,None,None,None,None,"MULTIPOLYGON (((-121.41303 39.73646, -121.4126..."
2,000423,2.40,0.86,0.0,False,2019-06-06,2019-06-06 14:30:18+00:00,2019-06-06 14:36:03+00:00,True,14:30:18,...,None,None,2019-06-06,P4A integrated camera,Map Pilot,None,None,None,None,"MULTIPOLYGON (((-121.41076 39.73592, -121.4106..."
3,000424,5.41,0.99,0.1,False,2019-06-25,2019-06-25 11:15:32+00:00,2019-06-25 13:23:21+00:00,True,11:15:32,...,None,None,2019-06-25,P4A integrated camera,Map Pilot,None,None,None,None,"MULTIPOLYGON (((-123.53096 40.33835, -123.5306..."
4,000425,3.01,0.95,0.1,False,2019-06-26,2019-06-26 11:56:23+00:00,2019-06-26 13:15:08+00:00,True,11:56:23,...,None,None,2019-06-26,P4A integrated camera,Map Pilot,None,None,None,None,"MULTIPOLYGON (((-123.52914 40.33610, -123.5290..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,000927,12.14,-0.32,45.0,True,2024-04-02,2024-04-02 17:16:15+00:00,2024-04-02 17:19:32+00:00,True,17:16:15,...,None,None,2024-04-02,M3E integrated RGB camera,None,None,None,None,False,"MULTIPOLYGON (((-122.14440 38.47535, -122.1438..."
436,000928,11.99,0.13,0.0,False,2024-04-02,2024-04-02 17:22:03+00:00,2024-04-02 17:25:09+00:00,True,17:22:03,...,None,None,2024-04-02,M3E integrated RGB camera,None,None,None,None,False,"MULTIPOLYGON (((-122.14734 38.47665, -122.1469..."
437,000929,9.87,-0.01,0.1,False,2024-04-02,2024-04-02 17:28:25+00:00,2024-04-02 17:32:16+00:00,True,17:28:25,...,None,None,2024-04-02,M3E integrated RGB camera,None,None,None,None,False,"MULTIPOLYGON (((-122.14737 38.47649, -122.1469..."
438,000930,10.82,0.07,0.1,False,2024-04-02,2024-04-02 17:37:00+00:00,2024-04-02 17:40:24+00:00,True,17:37:00,...,None,None,2024-04-02,M3E integrated RGB camera,None,None,None,None,False,"MULTIPOLYGON (((-122.14443 38.47527, -122.1439..."


In [9]:
# 3.3 We only keep missions where we have an ID and a datetime
gdf = gdf.dropna(subset=["dataset_id", "earliest_datetime_local_derived"])
gdf["id"] = gdf["dataset_id"].astype(str)

# Convert that local derived datetime string to a proper ISO 8601
gdf["datetime"] = pd.to_datetime(gdf["earliest_datetime_local_derived"])

# 3.4 Create empty lists to store STAC items and a place to track overall min/max for bounding box
all_items = []

# For temporal extent across the entire collection
all_datetimes = []

# We'll store numeric extremes for the global bounding box
min_x_list, min_y_list, max_x_list, max_y_list = [], [], [], []

In [11]:
gdf

,dataset_id,flight_speed_derived,flight_terrain_correlation_derived,camera_pitch_derived,smart_oblique_derived,earliest_date_derived,earliest_datetime_local_derived,latest_datetime_local_derived,single_date_derived,earliest_time_local_derived,...,date,sensor_name,flight_planner_name,addl_dataset_ids_baserow,addl_baserow_differ_by,why_not_separable,smart_oblique,geometry,id,datetime
0,000421,3.00,0.98,0.0,False,2019-06-04,2019-06-04 11:59:40+00:00,2019-06-04 14:31:00+00:00,True,11:59:40,...,2019-06-04,P4A integrated camera,Map Pilot,None,None,None,None,"MULTIPOLYGON (((-121.06144 40.13898, -121.0608...",000421,2019-06-04 11:59:40+00:00
1,000422,3.13,0.98,0.0,False,2019-06-06,2019-06-06 11:22:42+00:00,2019-06-06 14:30:16+00:00,True,11:22:42,...,2019-06-06,P4A integrated camera,Map Pilot,None,None,None,None,"MULTIPOLYGON (((-121.41303 39.73646, -121.4126...",000422,2019-06-06 11:22:42+00:00
2,000423,2.40,0.86,0.0,False,2019-06-06,2019-06-06 14:30:18+00:00,2019-06-06 14:36:03+00:00,True,14:30:18,...,2019-06-06,P4A integrated camera,Map Pilot,None,None,None,None,"MULTIPOLYGON (((-121.41076 39.73592, -121.4106...",000423,2019-06-06 14:30:18+00:00
3,000424,5.41,0.99,0.1,False,2019-06-25,2019-06-25 11:15:32+00:00,2019-06-25 13:23:21+00:00,True,11:15:32,...,2019-06-25,P4A integrated camera,Map Pilot,None,None,None,None,"MULTIPOLYGON (((-123.53096 40.33835, -123.5306...",000424,2019-06-25 11:15:32+00:00
4,000425,3.01,0.95,0.1,False,2019-06-26,2019-06-26 11:56:23+00:00,2019-06-26 13:15:08+00:00,True,11:56:23,...,2019-06-26,P4A integrated camera,Map Pilot,None,None,None,None,"MULTIPOLYGON (((-123.52914 40.33610, -123.5290...",000425,2019-06-26 11:56:23+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,000927,12.14,-0.32,45.0,True,2024-04-02,2024-04-02 17:16:15+00:00,2024-04-02 17:19:32+00:00,True,17:16:15,...,2024-04-02,M3E integrated RGB camera,None,None,None,None,False,"MULTIPOLYGON (((-122.14440 38.47535, -122.1438...",000927,2024-04-02 17:16:15+00:00
436,000928,11.99,0.13,0.0,False,2024-04-02,2024-04-02 17:22:03+00:00,2024-04-02 17:25:09+00:00,True,17:22:03,...,2024-04-02,M3E integrated RGB camera,None,None,None,None,False,"MULTIPOLYGON (((-122.14734 38.47665, -122.1469...",000928,2024-04-02 17:22:03+00:00
437,000929,9.87,-0.01,0.1,False,2024-04-02,2024-04-02 17:28:25+00:00,2024-04-02 17:32:16+00:00,True,17:28:25,...,2024-04-02,M3E integrated RGB camera,None,None,None,None,False,"MULTIPOLYGON (((-122.14737 38.47649, -122.1469...",000929,2024-04-02 17:28:25+00:00
438,000930,10.82,0.07,0.1,False,2024-04-02,2024-04-02 17:37:00+00:00,2024-04-02 17:40:24+00:00,True,17:37:00,...,2024-04-02,M3E integrated RGB camera,None,None,None,None,False,"MULTIPOLYGON (((-122.14443 38.47527, -122.1439...",000930,2024-04-02 17:37:00+00:00


In [11]:
# 3.5 Known asset filenames or patterns
#    We'll search for these in each mission's "processed/full" folder
TIF_FILENAMES = [
    "orthomosaic.tif", 
    "chm-mesh.tif", 
    "chm-ptcloud.tif", 
    "dsm-mesh.tif", 
    "dsm-ptcloud.tif", 
    "dtm-ptcloud.tif"
]
LAZ_FILENAMES = [
    "points.laz"
]

In [12]:
# 3.6 Iterate over each row in the GPKG
for idx, row in gdf.iterrows():
    mission_id = row["id"]  # e.g. "000421"
    mission_datetime = row["datetime"].tz_localize(None)  # remove time zone if any
    mission_folder = missions_root / mission_id

    # 3.6.1 Check if there's a subfolder that starts with "processed"
    #       Because the structure might be mission_id -> processed* -> full
    #       We'll do a quick search:
    if not mission_folder.exists():
        print(f"Mission folder does not exist: {mission_folder}. Skipping...")
        continue
    else:
        print(f"Mission folder found: {mission_folder}")
    
    #if not mission_folder.exists():
        # No such directory, skip
        #continue
    
    # Attempt to find path: mission_id/processed*/full
    processed_path = None
    for item in mission_folder.iterdir():
        if item.is_dir() and item.name.startswith("processed"):
            # We found something like processed or processed_whatever
            processed_path = item
            break
    
    if not processed_path:
        # No processed folder found, skip
        continue
    
    full_path = processed_path / "full"
    if not full_path.exists():
        # No full folder found
        continue

    # 3.6.2 Look for the known TIF or LAZ assets
    # We'll store data about each found asset in a dictionary
    assets_info = {}
    
    # Collect bounding geometries to union them
    union_poly = None
    union_bbox = [9999, 9999, -9999, -9999]  # [minx, miny, maxx, maxy]
    
    # TIFs
    for tif_name in TIF_FILENAMES:
        candidate_path = full_path / tif_name
        if candidate_path.exists():
            # Extract info
            tif_info = tif_get_spatial_info(candidate_path)
            if tif_info:
                # Update union bounding geometry
                minx, miny, maxx, maxy = tif_info["bbox"]
                
                # Expand union_bbox
                if minx < union_bbox[0]: union_bbox[0] = minx
                if miny < union_bbox[1]: union_bbox[1] = miny
                if maxx > union_bbox[2]: union_bbox[2] = maxx
                if maxy > union_bbox[3]: union_bbox[3] = maxy
                
                # We'll store the asset by a key that matches the file name, or something descriptive
                asset_key = tif_name.replace(".tif", "")
                assets_info[asset_key] = {
                    "href": tif_info["path"],
                    "type": tif_info["media_type"],
                    "roles": ["data"],  # Or something else 
                    "proj:epsg": tif_info["epsg"],
                    "gsd": [float(tif_info["x_res"]), float(tif_info["y_res"])],
                }
    
    # LAZ
    for laz_name in LAZ_FILENAMES:
        candidate_path = full_path / laz_name
        if candidate_path.exists():
            laz_info = las_get_spatial_info(candidate_path)
            if laz_info:
                minx, miny, maxx, maxy = laz_info["bbox"]
                if minx < union_bbox[0]: union_bbox[0] = minx
                if miny < union_bbox[1]: union_bbox[1] = miny
                if maxx > union_bbox[2]: union_bbox[2] = maxx
                if maxy > union_bbox[3]: union_bbox[3] = maxy
                
                asset_key = laz_name.replace(".laz", "")
                assets_info[asset_key] = {
                    "href": laz_info["path"],
                    "type": laz_info["media_type"],
                    "roles": ["data"],
                    "proj:epsg": laz_info["epsg"],
                    "gsd": [laz_info["x_res"], laz_info["y_res"]],
                }
    
    # If we found no assets, skip (per your requirement)
    if not assets_info:
        continue

    # 3.6.3 Build union geometry from union_bbox
    if union_bbox[0] == 9999 or union_bbox[1] == 9999:
        # Means we never updated it
        # Skip this mission if no bounding geometry
        continue
    
    # Create an item-level bounding box
    item_bbox = union_bbox  # [minX, minY, maxX, maxY]

    # Create a polygon footprint
    item_footprint = Polygon([
        (item_bbox[0], item_bbox[1]),  # left, bottom
        (item_bbox[0], item_bbox[3]),  # left, top
        (item_bbox[2], item_bbox[3]),  # right, top
        (item_bbox[2], item_bbox[1])   # right, bottom
    ])
    
    # 3.6.4 Update global bounding box and time
    min_x_list.append(item_bbox[0])
    min_y_list.append(item_bbox[1])
    max_x_list.append(item_bbox[2])
    max_y_list.append(item_bbox[3])
    all_datetimes.append(mission_datetime)

    # 3.6.5 Build the STAC Item dictionary 
    #       If you prefer raw dict approach (rather than PySTAC):
    item_dict = {
        "type": "Feature",
        "collection": "Drone Missions Collection",
        "stac_extensions": [
                "https://stac-extensions.github.io/projection/v1.0.0/schema.json",
                "https://stac-extensions.github.io/scientific/v1.0.0/schema.json"
        ],
        "stac_version": "1.0.0",
        "id": mission_id,
        "properties": {
            "datetime": mission_datetime.isoformat(),  # or string
        },
        "geometry": mapping(item_footprint),  # geojson geometry
        "bbox": item_bbox,
        "links": [],  # can add if needed
        "assets": assets_info
    }
    # Add license, or you can place it at collection-level only. 
    # But let's keep consistent with your specification
    item_dict["properties"]["license"] = "CC BY 4.0"
    
    all_items.append(item_dict)

Mission folder found: /data-store/iplant/home/shared/ofo/public/missions/000421
Mission folder found: /data-store/iplant/home/shared/ofo/public/missions/000422
Mission folder found: /data-store/iplant/home/shared/ofo/public/missions/000423
Mission folder found: /data-store/iplant/home/shared/ofo/public/missions/000424
Mission folder found: /data-store/iplant/home/shared/ofo/public/missions/000425
Mission folder found: /data-store/iplant/home/shared/ofo/public/missions/000426
Mission folder found: /data-store/iplant/home/shared/ofo/public/missions/000427
Mission folder found: /data-store/iplant/home/shared/ofo/public/missions/000428
Mission folder found: /data-store/iplant/home/shared/ofo/public/missions/000429
Mission folder found: /data-store/iplant/home/shared/ofo/public/missions/000430
Mission folder found: /data-store/iplant/home/shared/ofo/public/missions/000431
Mission folder found: /data-store/iplant/home/shared/ofo/public/missions/000432
Mission folder found: /data-store/iplant

In [13]:
print(all_items)

[{'type': 'Feature', 'collection': 'Drone Missions Collection', 'stac_extensions': ['https://stac-extensions.github.io/projection/v1.0.0/schema.json', 'https://stac-extensions.github.io/scientific/v1.0.0/schema.json'], 'stac_version': '1.0.0', 'id': '000547', 'properties': {'datetime': '2020-05-12T13:40:27', 'license': 'CC BY 4.0'}, 'geometry': {'type': 'Polygon', 'coordinates': (((-121.72742048571602, 37.39116775238257), (-121.72742048571602, 37.395396317847), (-121.72226705269746, 37.395396317847), (-121.72226705269746, 37.39116775238257), (-121.72742048571602, 37.39116775238257)),)}, 'bbox': [-121.72742048571602, 37.39116775238257, -121.72226705269746, 37.395396317847], 'links': [], 'assets': {'orthomosaic': {'href': '/data-store/iplant/home/shared/ofo/public/missions/000547/processed-20241218T1059/full/orthomosaic.tif', 'type': <MediaType.COG: 'image/tiff; application=geotiff; profile=cloud-optimized'>, 'roles': ['data'], 'proj:epsg': 3310, 'gsd': [0.055, 0.055]}, 'chm-mesh': {'hre

In [14]:
# -----------------------------
# 4. Build the STAC Collection
# -----------------------------
if len(all_items) == 0:
    print("No STAC Items found! Exiting with no outputs.")
    # You can decide if you want to write empty files or not
    raise SystemExit

collection_bbox = [
    float(np.min(min_x_list)), 
    float(np.min(min_y_list)), 
    float(np.max(max_x_list)), 
    float(np.max(max_y_list))
]

# For temporal extent
min_date = min(all_datetimes)
max_date = max(all_datetimes)

collection_dict = {
    "type": "Collection",
    "stac_version": "1.0.0",
    "id": "Drone Missions Collection",
    "description": "STAC collection for drone missions polygons.",
    "license": "CC BY 4.0",
    "links": [],
    "extent": {
        "spatial": {
            "bbox": [collection_bbox]
        },
        "temporal": {
            "interval": [[
                min_date.isoformat(),
                max_date.isoformat()
            ]]
        }
    }
}

In [15]:
# -----------------------------
# 5. Export to JSON
# -----------------------------

# 5.1 index.geojson: FeatureCollection of Items
index_geojson = {
    "type": "FeatureCollection",
    "features": all_items
}

with open("/data-store/iplant/home/jgillan/index4.geojson", "w", encoding="utf-8") as f:
    json.dump(index_geojson, f, indent=2)

# 5.2 collection.json
with open("/data-store/iplant/home/jgillan/collection4.json", "w", encoding="utf-8") as f:
    json.dump(collection_dict, f, indent=2)

print("STAC Collection (collection.json) and Items (index.geojson) have been created!")


STAC Collection (collection.json) and Items (index.geojson) have been created!


In [ ]:
blah